In [ ]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from numpy import matrix
import pickle
from sklearn.metrics import mean_squared_error
from get_timbre import get_timbre
from evaluation import evaluate_data

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'

In [8]:
playlist_ids = ['5odVaQ10ISGhvuakjVGmxp', '0mCV9EHiVXEsbw307lBF25', '4VGvI1dODxGyBop5jFhgfN',
                '5pVRL48wvxNRL8lWkxdVCX', '5Kq61WYIbYGLB6dvuTKzZV', '3MO4sQXwvB13TAdiSoG9AR',
                '64iSau8h74DkOUG8JVY3W2', '5ZWw4qko5ccxnEIA8blVSq', '35iVxmwsHyJVSdvRasuK8V',
                '06LB9aYKvibLEXb4Qf3gHM', '2dfxSJXVFYpZi2YOjgbwRX', '6FwDZbpBnGXBOJ1UwW9lKb',
                '44vN1eprBxUdirtb9pCOge', '64mSbvFQYIW547z4zWrEQK', '3VBWC197hTEmayqbDTmvp4',
                '2Am8cD0icf62d4dL7NrDTx', '0fjjY976NVJ03tPhxjamcX', '0vzdOGXJBf6SBFeYbwfpnL',
                '5JufAIzUR25lgE8zrums4k', '2ppGD3RwccxuPSJZiyZrCm', '4LBHOUvGlk4RtaMbfB5huK',
                '1VJZKBsulYCry6QpSoW5mG', '0dnsYgOOtfsOg78qLrZEAv', '5KtPLkVfM18fVQ4puapK98',
                '51eXnQHgNcFw7xGllU5kCf', '23x3CI4Cncf6BpSnIeZlF7', '1rDhjtHJkAIlUWgJlxQSRF',
                '5cS1vW9gzr2lLFZ0XpkmfD', '2xxXRaeMZSoAiUslTqbB7S', '0Gx1JfJVTkWSfCcTkMgJiA',
                ]

In [ ]:
x = 1
for playlist_id in playlist_ids:
    print('Running playlist ' + playlist_id)

    # Get track data
    
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    track_ids = []
    for track in tracks:
        track_ids.append(track['track']['id'])

    columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
    headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                'id', 'duration_ms', 'time_signature', 'track_title', 'album_title',
                'album_artist', 'track_number', 'total_tracks']

    playlist_data = pd.DataFrame(columns=headings)
    playlist_length = len(track_ids)

    i = 0
    for track in track_ids:
        results = sp.audio_features(track)
        if results[0]: 
            features = results[0]
        
        features_matrix = pd.DataFrame.from_records(features, index=[0])
        
        # Remove unneeded columns
        features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

        # Add track info
        features_matrix['track_title'] = tracks[i]['track']['name']
        features_matrix['album_title'] = tracks[i]['track']['album']['name']
        features_matrix['album_artist'] = tracks[i]['track']['album']['artists'][0]['name']

        # Add track number and total tracks
        features_matrix['track_number'] = tracks[i]['track']['track_number']
        features_matrix['total_tracks'] = playlist_length

        playlist_data = pd.concat([playlist_data, features_matrix])
        i += 1

    playlist_data = playlist_data.reset_index(drop=True)
    playlist_data_full = pd.DataFrame(playlist_data)

    # Get timbre data

    timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

    for track in track_ids:
        print('Getting timbre for: '+ track)
        track_timbre = get_timbre(track, playlist_data_full)
        timbre = pd.concat([timbre, track_timbre], axis=0)

    timbre = timbre.reset_index(drop=True)
    playlist_data = pd.concat([playlist_data, timbre], axis=1)

    playlist_data.to_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data_full.to_csv('./data/test_set/playlist_data_full_' + str(x) + '.csv')
    x += 1

In [ ]:
x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []

for playlist_id in playlist_ids:
    
    playlist_data.to_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data_full.to_csv('./data/test_set/playlist_data_full_' + str(x) + '.csv')
    x += 1

    print('Running playlist ' + playlist_id)
    # Run model on data

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(matrix(original_order_eval))

    # Load and run trained model
    model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(playlist_data.values)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(matrix(sorted_order_eval))

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

print(mean_squared_errors)

In [ ]:
changes = []

for eval in sorted_evals:
    changes.append(sorted_evals[eval] - original_evals[eval])
    
print(changes)

In [ ]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST